In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings

matplotlib.rcParams['font.family'] = 'Binggrae'
matplotlib.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv', index_col = 'index')
df_test = pd.read_csv('test.csv', index_col = 'index')

In [3]:
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
index,,,,
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,1129000014045300,5011000263065200,스마트디바이스,6


In [4]:
df_test

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
index,,,
0,4167000577042200,5011000435014100,선케어
1,1156000009012200,5011000172034400,구강위생용품
2,4122000363057300,5011000361097300,캠핑
3,5011000436041400,2826000084036400,아웃도어가구
4,4150000241065200,5011000169044300,분유/이유식/아기간식
...,...,...,...
7915,5011000266051200,4623000417038100,농산물
7916,1154500001098300,5011000264055100,문화컨텐츠
7917,5013000610049100,1147000018091400,농산물


## EDA

1. 송하량과 수하량을 따로 분류해서 받아야 함
2. 격자공간고유번호 별로 총 건수를 잡아야 함
3. 너무 많은 격자공간이 있으므로 시군구별로 받아서 분출
4. 카테고리가 너무 다수, 좀 더 줄일 수 있도록 해야 함

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   송하인_격자공간고유번호  31684 non-null  int64 
 1   수하인_격자공간고유번호  31684 non-null  int64 
 2   물품_카테고리       31684 non-null  object
 3   운송장_건수        31684 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


In [6]:
df_train = df_train.astype({'운송장_건수':int, '송하인_격자공간고유번호':str,'수하인_격자공간고유번호':str})
df_train['송하인_시군구코드'] = df_train['송하인_격자공간고유번호'].str[:5]
df_train['수하인_시군구코드'] = df_train['수하인_격자공간고유번호'].str[:5]
# df_train = df_train.drop(columns = '수하인_격자공간고유번호').groupby('송하인_격자공간고유번호').sum()
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_시군구코드,수하인_시군구코드
index,,,,,,
0,5011000595017300,2871000192069300,음반,3,50110,28710
1,4148000690043300,5011000264024400,문화컨텐츠,3,41480,50110
2,5011000078068400,1120000007005400,농산물,3,50110,11200
3,4127100048006400,5011000587019400,기타식품,7,41271,50110
4,5011000078068400,2823700010076300,농산물,3,50110,28237
...,...,...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3,44710,50110
31680,1129000014045300,5011000319087100,스마트디바이스,4,11290,50110
31681,1129000014045300,5011000263065200,스마트디바이스,6,11290,50110


In [7]:
df=df_train.pivot_table(index = ['송하인_시군구코드','물품_카테고리'], values = '운송장_건수', aggfunc = sum)
df['송하인_시군구코드'] = df.index
df = df.reset_index(drop = True)
df['송하인_시군구코드'] = df['송하인_시군구코드'].astype(str)
l=df['송하인_시군구코드'].str.split(',')
df['카테고리'] =l.str.get(1)
df['송하인_시군구코드'] =l.str.get(0)
df['카테고리'] = df['카테고리'].str.replace(pat=r"['')]", repl=r"", regex=True)
df['송하인_시군구코드'] = df['송하인_시군구코드'].str.replace(pat=r"[''(]", repl=r"", regex=True)
df

,운송장_건수,송하인_시군구코드,카테고리
0,8,11110,가공식품
1,5,11110,건강식품
2,42,11110,공구
3,49,11110,과자
4,5,11110,기타식품
...,...,...,...
2337,170,50130,축산
2338,35,50130,취미용품
2339,26,50130,커튼/블라인드
2340,10,50130,하의


In [8]:
code = pd.read_excel('도로명코드_전체.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구 코드  267 non-null    int64 
 1   시도 이름   267 non-null    object
 2   시군구 이름  266 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.4+ KB


In [9]:
code['시군구 명칭'] = code['시도 이름'] +" " + code['시군구 이름']
code.drop(columns = ['시도 이름','시군구 이름'], inplace = True)
code=code.astype({'시군구 코드':str})
code

,시군구 코드,시군구 명칭
0,11110,서울특별시 종로구
1,11140,서울특별시 중구
2,11170,서울특별시 용산구
3,11200,서울특별시 성동구
4,11215,서울특별시 광진구
...,...,...
262,44130,충청남도 천안시
263,44730,충청남도 연기군
264,44830,충청남도 당진군
265,45110,전라북도 전주시


In [10]:
code.loc[267] =['36110','세종특별자치시']
# data_to_insert = {'시군구 코드': '36110', '시군구 명칭': '세종특별자치시'}

In [11]:
code

,시군구 코드,시군구 명칭
0,11110,서울특별시 종로구
1,11140,서울특별시 중구
2,11170,서울특별시 용산구
3,11200,서울특별시 성동구
4,11215,서울특별시 광진구
...,...,...
263,44730,충청남도 연기군
264,44830,충청남도 당진군
265,45110,전라북도 전주시
266,48120,경상남도 창원시


In [12]:
S = pd.merge(df, code, how = 'left', left_on = '송하인_시군구코드', right_on = '시군구 코드')
S['송하건수'] = S['운송장_건수']
S = S.drop(columns = ['시군구 코드','운송장_건수'])
S

,송하인_시군구코드,카테고리,시군구 명칭,송하건수
0,11110,가공식품,서울특별시 종로구,8
1,11110,건강식품,서울특별시 종로구,5
2,11110,공구,서울특별시 종로구,42
3,11110,과자,서울특별시 종로구,49
4,11110,기타식품,서울특별시 종로구,5
...,...,...,...,...
2343,50130,축산,제주특별자치도 서귀포시,170
2344,50130,취미용품,제주특별자치도 서귀포시,35
2345,50130,커튼/블라인드,제주특별자치도 서귀포시,26
2346,50130,하의,제주특별자치도 서귀포시,10


In [13]:
dft=df_train.pivot_table(index = ['수하인_시군구코드','물품_카테고리'], values = '운송장_건수', aggfunc = sum)
dft['수하인_시군구코드'] = dft.index
dft = dft.reset_index(drop = True)
dft['수하인_시군구코드'] = dft['수하인_시군구코드'].astype(str)
l=dft['수하인_시군구코드'].str.split(',')
dft['카테고리'] =l.str.get(1)
dft['수하인_시군구코드'] =l.str.get(0)
dft['카테고리'] = dft['카테고리'].str.replace(pat=r"['')]", repl=r"", regex=True)
dft['수하인_시군구코드'] = dft['수하인_시군구코드'].str.replace(pat=r"[''(]", repl=r"", regex=True)
dft

,운송장_건수,수하인_시군구코드,카테고리
0,11,11110,가공식품
1,3,11110,건강식품
2,18,11110,과자
3,13,11110,기타식품
4,14,11110,기타패션의류
...,...,...,...
1833,3,50130,헤어스타일링
1834,4,50130,헤어액세서리
1835,77,50130,헤어케어
1836,7,50130,헬스


In [14]:
R = pd.merge(dft, code, how = 'left', left_on = '수하인_시군구코드', right_on = '시군구 코드')
R['수하건수'] = R['운송장_건수']
R = R.drop(columns = ['시군구 코드','운송장_건수'])
R

,수하인_시군구코드,카테고리,시군구 명칭,수하건수
0,11110,가공식품,서울특별시 종로구,11
1,11110,건강식품,서울특별시 종로구,3
2,11110,과자,서울특별시 종로구,18
3,11110,기타식품,서울특별시 종로구,13
4,11110,기타패션의류,서울특별시 종로구,14
...,...,...,...,...
1842,50130,헤어스타일링,제주특별자치도 서귀포시,3
1843,50130,헤어액세서리,제주특별자치도 서귀포시,4
1844,50130,헤어케어,제주특별자치도 서귀포시,77
1845,50130,헬스,제주특별자치도 서귀포시,7


In [15]:
R[R['시군구 명칭'].isna()]

,수하인_시군구코드,카테고리,시군구 명칭,수하건수
645,36110,가공식품,NaN,7
647,36110,과자,NaN,3
649,36110,기타식품,NaN,10
651,36110,농산물,NaN,517
653,36110,문화컨텐츠,NaN,11
655,36110,수산,NaN,30
657,36110,음료,NaN,8
659,36110,음반,NaN,7
661,36110,축산,NaN,44


In [16]:
S[S['시군구 명칭'].isna()]

,송하인_시군구코드,카테고리,시군구 명칭,송하건수
850,36110,가공식품,NaN,3
852,36110,농산물,NaN,7
854,36110,스킨케어,NaN,9
856,36110,신발,NaN,6
858,36110,위생/건강용품,NaN,3
860,36110,주방용품,NaN,3


In [17]:
R.loc[R['시군구 명칭'].isna(),'시군구 명칭'] = '세종특별자치시'

In [18]:
S.loc[S['시군구 명칭'].isna(),'시군구 명칭'] = '세종특별자치시'

In [19]:
df = pd.merge(R,S,how = 'outer', on = '시군구 명칭' )
df['수하인_카테고리'] = df['카테고리_x']
df['송하인_카테고리'] = df['카테고리_y']
df.drop(columns = ['카테고리_x','카테고리_y'], inplace = True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29775 entries, 0 to 29774
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   수하인_시군구코드  29775 non-null  object 
 1   시군구 명칭     29775 non-null  object 
 2   수하건수       29775 non-null  int32  
 3   송하인_시군구코드  29732 non-null  object 
 4   송하건수       29732 non-null  float64
 5   수하인_카테고리   29775 non-null  object 
 6   송하인_카테고리   29732 non-null  object 
dtypes: float64(1), int32(1), object(5)
memory usage: 1.7+ MB


In [21]:
df['송하건수'].fillna(value = 0, inplace = True)

In [22]:
df['송하건수'] = df['송하건수'].astype(int)

In [23]:
df[df['송하인_카테고리'].isna()]

,수하인_시군구코드,시군구 명칭,수하건수,송하인_시군구코드,송하건수,수하인_카테고리,송하인_카테고리
7464,28720,인천광역시 옹진군,11,NaN,0,농산물,NaN
7465,28720,인천광역시 옹진군,5,NaN,0,음료,NaN
8313,31110,울산광역시 중구,3,NaN,0,가공식품,NaN
8314,31110,울산광역시 중구,3,NaN,0,건강식품,NaN
8315,31110,울산광역시 중구,3,NaN,0,기타식품,NaN
8316,31110,울산광역시 중구,254,NaN,0,농산물,NaN
8317,31110,울산광역시 중구,6,NaN,0,수산,NaN
8318,31110,울산광역시 중구,9,NaN,0,음료,NaN
8319,31110,울산광역시 중구,7,NaN,0,축산,NaN
17325,41830,경기도 양평군,31,NaN,0,과자,NaN


In [24]:
df['송하인_시군구코드'].fillna(value = 'None', inplace = True)
df['송하인_카테고리'].fillna(value = 'None', inplace = True)

In [25]:
df.sort_values('수하건수', ascending = False)

,수하인_시군구코드,시군구 명칭,수하건수,송하인_시군구코드,송하건수,수하인_카테고리,송하인_카테고리
22920,50110,제주특별자치도 제주시,3981,50110,10,문화컨텐츠,뷰티소품
22910,50110,제주특별자치도 제주시,3981,50110,41205,문화컨텐츠,농산물
22951,50110,제주특별자치도 제주시,3981,50110,14,문화컨텐츠,홈데코
22921,50110,제주특별자치도 제주시,3981,50110,36,문화컨텐츠,상의
22919,50110,제주특별자치도 제주시,3981,50110,3,문화컨텐츠,분유/이유식/아기간식
...,...,...,...,...,...,...,...
17398,42110,강원도 춘천시,3,42110,34,기타화장품/미용,분유/이유식/아기간식
17397,42110,강원도 춘천시,3,42110,3,기타화장품/미용,반려동물
11463,41310,경기도 구리시,3,41310,4,음료,가공식품
11464,41310,경기도 구리시,3,41310,11,음료,건강식품


In [26]:
df.isnull().sum()

수하인_시군구코드    0
시군구 명칭       0
수하건수         0
송하인_시군구코드    0
송하건수         0
수하인_카테고리     0
송하인_카테고리     0
dtype: int64

## Linear _송하

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [41]:
df_S = df[['시군구 명칭','송하건수','송하인_카테고리']]

In [42]:
df_R = df[['시군구 명칭','수하건수','수하인_카테고리']]

In [49]:
ct = ColumnTransformer([
('onehot',OneHotEncoder(sparse=False),['시군구 명칭','송하인_카테고리'])])
#CT 이용 시에는 튜플형태로 

In [101]:
df_train_y = df_S['송하건수']
df_train_x = df_S.drop(columns = '송하건수')

ct.fit(df_train_x)
df_train_x=ct.transform(df_train_x)

X_train,X_test,y_train,y_test=train_test_split(df_train_x,df_train_y,random_state=2022)

X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [102]:
X_train.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [62]:
!pip install xgboost 

In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from math import sqrt

In [68]:
pipe = Pipeline([('regressor', RandomForestRegressor())])
search_space = [
                {
                 'regressor': [RandomForestRegressor(random_state=0)],
                 'regressor__n_estimators': [100, 150, 200],
                 'regressor__max_depth': [3, 4, 5]
                },
                
                {
                 'regressor': [XGBRegressor()],
                 'regressor__max_depth': [3],
                 'regressor__n_estimators': [300],
                 'regressor__eta': [0.02, 0.1],
                 'regressor__subsample': [0.5, 0.6],
                 'regressor__seed':[0]
                },
                
                {
                 'regressor': [DecisionTreeRegressor(random_state=0)],
                 'regressor__min_samples_split':[3, 4, 5],
                 'regressor__max_depth': [3, 4, 5]
                }
               ]

In [69]:
kf = KFold(n_splits=9,random_state=None)
grid_search = GridSearchCV(pipe,search_space, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train.values.ravel())

print('best params:', grid_search.best_params_)
print('best estimator:', grid_search.best_estimator_)
print('best rmse :', sqrt(-(grid_search.best_score_)))

best params: {'regressor': XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             enable_categorical=False, eta=0.1, gamma=None, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, seed=0, subsample=0.6, tree_method=None,
             validate_parameters=None, verbosity=None), 'regressor__eta': 0.1, 'regressor__max_depth': 3, 'regressor__n_estimators': 300, 'regressor__seed': 0, 'regressor__subsample': 0.6}
best estimator: Pipeline(steps=[('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_

In [112]:
model = XGBRegressor(max_dept = 5 , n_estimators = 300 , seed = 0 )
model.fit(X_train, y_train)
print(model.score(X_train, y_train))



[11:37:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_dept" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


0.9999973232347572


In [105]:
y_pred=model.predict(X_test)
MSE = mean_squared_error(y_test, y_pred) 
print('RMSE:',np.sqrt(MSE))

RMSE: 5.934609714380243


In [108]:
X_test.shape

(7444, 352)

In [109]:
X_train.shape

(22331, 352)